In [1]:
import numpy as np
from qutip import basis, ket2dm, tensor, expect, fidelity, qeye

# Define the states
H = basis(2,0)
V = basis(2,1)

## Maximally entangled EPR pairs:
phi_plus = (tensor(H, H) + tensor(V, V)).unit()

## Maximally entangled EPR pairs with mixture:
rho = 0.7*ket2dm(phi_plus) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

## Non-maximally entangled pair:
phi = (1/np.sqrt(5)*tensor(H,H) + 2/np.sqrt(5)*tensor(V,V)).unit()


## Non-maximally entangled pair with mixture:
rho_prime = 0.7*ket2dm(phi) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

c:\Users\tvle2\AppData\Local\anaconda3\envs\QT\lib\site-packages\qutip\__init__.py:24: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


### Simulation of photon generation

In [2]:
def simulate_detection(rate, eff, dark_rate, dead_time, totol_time):
    t, counts = 0, 0
    dead = False
    while t < totol_time:
        if dead:
            t+= dead_time
            dead = False
        else:  
            photon_interval = np.random.exponential(1/rate)
            dark_interval = np.random.exponential(1/dark_rate)
            dt = min(photon_interval, dark_interval)
            t += dt
            if np.random.rand() < eff:
                counts += 1
                dead = True
    return counts

### Basis selection and Bell measurement

In [3]:
proj = lambda theta: (np.cos(theta)*H + np.sin(theta)*V).unit()

def compute_expectation(rho, theta_a, theta_b):
    A = ket2dm(proj(theta_a))
    B = ket2dm(proj(theta_b))
    AB = tensor(A, B)
    E = expect(AB, rho) + expect(tensor(qeye(2) - A, qeye(2) - B), rho) - \
        expect(tensor(A, qeye(2) - B), rho) - expect(tensor(qeye(2) - A, B), rho)
    return E

### Compute metrics
- Total Count Rate: Count all photons detected (including dark counts) per second
- Coincidence Rate: Count events detected simultaneously (within 1ns window)
- Fidelity: Calculate using QuTip
- Bell S parameter

In [4]:
states = {'Phi+': phi_plus, 'rho': rho, 'phi': phi, 'rho_prime': rho_prime}
results = {}

for name, state in states.items():
    counts_A = simulate_detection(15000, 0.1, 1000, 4e-6, 30)
    counts_B = simulate_detection(15000, 0.1, 1000, 4e-6, 30)

    coincidence_rate = min(counts_A, counts_B) /30

    F = fidelity(state, ket2dm(phi_plus))
    E1 = compute_expectation(state, 0, np.pi/8) # 0, 22.5
    E2 = compute_expectation(state, 0, 3*np.pi/8) # 0, 67.5
    E3 = compute_expectation(state, np.pi/4, np.pi/8) # 45, 22.5
    E4 = compute_expectation(state, np.pi/4, 3*np.pi/8) # 45, 67.5

    S = E1 - E2 + E3 + E4

    results[name] = {
        'Total Counts (Alice)': counts_A/30,
        'Totol Counts (Bob)': counts_B/30,
        'Coincidence Rate': coincidence_rate,
        'Fidelity': F,
        'Bell S Value': S
    }

print(results)

{'Phi+': {'Total Counts (Alice)': 1595.2, 'Totol Counts (Bob)': 1582.2666666666667, 'Coincidence Rate': 1582.2666666666667, 'Fidelity': np.float64(0.9999999999999997), 'Bell S Value': 2.82842712474619}, 'rho': {'Total Counts (Alice)': 1594.6, 'Totol Counts (Bob)': 1591.3333333333333, 'Coincidence Rate': 1591.3333333333333, 'Fidelity': np.float64(0.8366600265340758), 'Bell S Value': 1.5556349186104041}, 'phi': {'Total Counts (Alice)': 1584.4333333333334, 'Totol Counts (Bob)': 1577.1666666666667, 'Coincidence Rate': 1577.1666666666667, 'Fidelity': np.float64(0.9486832980505141), 'Bell S Value': 2.5455844122715723}, 'rho_prime': {'Total Counts (Alice)': 1601.2, 'Totol Counts (Bob)': 1581.8, 'Coincidence Rate': 1581.8, 'Fidelity': np.float64(0.7937253933193772), 'Bell S Value': 1.3576450198781722}}


In [9]:
import numpy as np
from qutip import basis, ket2dm, tensor, fidelity, qeye

###############################################################################
# 1) Quantum states (same as before)
###############################################################################
H = basis(2, 0)
V = basis(2, 1)

phi_plus = (tensor(H, H) + tensor(V, V)).unit()
rho = 0.7*ket2dm(phi_plus) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

phi = (1/np.sqrt(5)*tensor(H,H) + 2/np.sqrt(5)*tensor(V,V)).unit()
rho_prime = 0.7*ket2dm(phi) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

all_states = {
    "Phi+": phi_plus,
    "rho": rho,
    "phi": phi,
    "rho_prime": rho_prime
}

###############################################################################
# 2) Generate entangled pair arrival times
#    Now there's ONE Poisson process for the entire source.
###############################################################################
def generate_pair_timestamps(pair_rate, total_time):
    """
    Return an array of 'pair' arrival times (in seconds).
    The source creates entangled pairs at a Poisson average of 'pair_rate'.
    """
    # total number of pairs in total_time ~ Poisson(mean = pair_rate*total_time)
    n_pairs = np.random.poisson(lam=pair_rate * total_time)
    # we place them randomly (uniformly) in [0, total_time] or we can do an
    # incremental approach. We'll do uniform for simplicity:
    return np.random.rand(n_pairs) * total_time

###############################################################################
# 3) Local detection logic for each side
###############################################################################
def detect_photons_from_pairs(pair_times,
                              eff_detector,
                              dead_time,
                              total_time):
    """
    For a list of 'pair_times' that are sent to, say, Bob,
    we simulate detection of the "Bob half" of each pair with a 2-step approach:

      1) Sort pair_times.
      2) Walk through them in chronological order:
         - If the time since last detection < dead_time, skip (still 'dead').
         - Otherwise, detect with probability = eff_detector.
           If we detect, record that time in an array of 'photon detections'.

    Returns: np.array of detection times for the real photons
    """
    times_sorted = np.sort(pair_times)
    detect_times = []
    last_det_t = -np.inf
    for t in times_sorted:
        if t - last_det_t < dead_time:
            continue
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

def generate_dark_counts(dark_rate, eff_detector, dead_time, total_time):
    """
    Separate function that simulates the arrival of dark counts for total_time
    (Poisson process) and applies detection logic with efficiency & dead time.

    Returns: np.array of detection times for the dark counts.
    """
    # 1) Generate times for dark "arrivals"
    n_dark = np.random.poisson(dark_rate * total_time)
    dark_arrivals = np.random.rand(n_dark) * total_time
    dark_arrivals.sort()

    detect_times = []
    last_det_t = -np.inf
    for t in dark_arrivals:
        if t - last_det_t < dead_time:
            continue
        # dark arrival "arrived" -> do we detect it?
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t

    return np.array(detect_times)

###############################################################################
# 4) Counting coincidences with separate categories: photon-photon, photon-dark...
###############################################################################
def count_coincidences(tA_photons, tA_dark, tB_photons, tB_dark, coincidence_window=1e-9):
    """
    Return a dictionary with:
      - 'photon-photon': number of coincidences between tA_photons & tB_photons
      - 'photon-dark':   etc...
      - 'dark-photon':
      - 'dark-dark':

    We'll do a standard function for pairwise coincidence counting of two
    sorted arrays, then apply it to each pair of arrays.
    """
    results = {}
    results["photon-photon"] = _count_coincidences_2arr(tA_photons, tB_photons, coincidence_window)
    results["photon-dark"]   = _count_coincidences_2arr(tA_photons, tB_dark, coincidence_window)
    results["dark-photon"]   = _count_coincidences_2arr(tA_dark, tB_photons, coincidence_window)
    results["dark-dark"]     = _count_coincidences_2arr(tA_dark, tB_dark, coincidence_window)
    return results

def _count_coincidences_2arr(times1, times2, window):
    """
    Standard two-pointer approach. Count how many pairs of (t1, t2)
    lie within +/- 'window' of each other.
    times1, times2 are sorted arrays.
    """
    i, j = 0, 0
    n_coinc = 0
    while i < len(times1) and j < len(times2):
        dt = times1[i] - times2[j]
        if abs(dt) <= window:
            n_coinc += 1
            i += 1
            j += 1
        elif dt < 0:
            i += 1
        else:
            j += 1
    return n_coinc

###############################################################################
# 5) Compute the CHSH S-value from the density matrix (unchanged)
###############################################################################
def projector(theta):
    st = np.cos(theta)*H + np.sin(theta)*V
    return ket2dm(st.unit())

def compute_expectation(rho_2q, theta_a, theta_b):
    A_plus = projector(theta_a)
    B_plus = projector(theta_b)

    A_minus = qeye(2) - A_plus
    B_minus = qeye(2) - B_plus

    P_pp = tensor(A_plus, B_plus)
    P_mm = tensor(A_minus, B_minus)
    P_pm = tensor(A_plus, B_minus)
    P_mp = tensor(A_minus, B_plus)

    p_pp = (P_pp * rho_2q).tr().real
    p_mm = (P_mm * rho_2q).tr().real
    p_pm = (P_pm * rho_2q).tr().real
    p_mp = (P_mp * rho_2q).tr().real

    return (p_pp + p_mm) - (p_pm + p_mp)

###############################################################################
# 6) Putting it all together in a main routine
###############################################################################
def main():
    # Simulation parameters
    PAIR_RATE    = 15000  # pairs per second
    DARK_RATE    = 1000   # dark counts per second
    EFF_DETECTOR = 0.10
    DEAD_TIME    = 4e-6
    TOTAL_TIME   = 30
    COINC_WINDOW = 1e-9

    results = {}

    for state_name, rho_state in all_states.items():
        #--- A) Generate entangled pair arrival times (shared for Alice+Bob)
        pair_arrivals = generate_pair_timestamps(PAIR_RATE, TOTAL_TIME)

        #--- B) For each side, detect "photons from the source"
        a_photon_times = detect_photons_from_pairs(pair_arrivals, EFF_DETECTOR, DEAD_TIME, TOTAL_TIME)
        b_photon_times = detect_photons_from_pairs(pair_arrivals, EFF_DETECTOR, DEAD_TIME, TOTAL_TIME)

        #--- C) Generate + detect dark counts on each side
        a_dark_times = generate_dark_counts(DARK_RATE, EFF_DETECTOR, DEAD_TIME, TOTAL_TIME)
        b_dark_times = generate_dark_counts(DARK_RATE, EFF_DETECTOR, DEAD_TIME, TOTAL_TIME)

        #--- D) Merge them if you want total detection times
        a_all = np.sort(np.concatenate((a_photon_times, a_dark_times)))
        b_all = np.sort(np.concatenate((b_photon_times, b_dark_times)))

        #--- E) Coincidence counting with 1 ns window
        cdict = count_coincidences(a_photon_times, a_dark_times,
                                   b_photon_times, b_dark_times,
                                   coincidence_window=COINC_WINDOW)

        # We can compute a "total" coincidence from a_all and b_all
        total_coinc = _count_coincidences_2arr(a_all, b_all, COINC_WINDOW)

        #--- F) Rates
        alice_total_rate = len(a_all) / TOTAL_TIME
        bob_total_rate   = len(b_all) / TOTAL_TIME
        coincidence_rate = total_coinc / TOTAL_TIME

        #--- G) Fidelity wrt. |Phi+>
        F = fidelity(rho_state, ket2dm(phi_plus))

        #--- H) Bell S from the density matrix (theoretical)
        E1 = compute_expectation(rho_state, 0,       np.pi/8)
        E2 = compute_expectation(rho_state, 0,       3*np.pi/8)
        E3 = compute_expectation(rho_state, np.pi/4, np.pi/8)
        E4 = compute_expectation(rho_state, np.pi/4, 3*np.pi/8)
        S = E1 - E2 + E3 + E4

        #--- I) Store in results
        results[state_name] = {
            "Alice total rate (Hz)": alice_total_rate,
            "Bob total rate (Hz)":   bob_total_rate,

            "Alice photon rate": len(a_photon_times)/TOTAL_TIME,
            "Alice dark rate":   len(a_dark_times)/TOTAL_TIME,
            "Bob photon rate":   len(b_photon_times)/TOTAL_TIME,
            "Bob dark rate":     len(b_dark_times)/TOTAL_TIME,

            "Coincidences (ph-ph)": cdict["photon-photon"],
            "Coincidences (ph-dark)": cdict["photon-dark"],
            "Coincidences (dark-ph)": cdict["dark-photon"],
            "Coincidences (dark-dark)": cdict["dark-dark"],
            "Total coincidences": total_coinc,
            "Coincidence rate (Hz)": coincidence_rate,

            "Fidelity vs PhiPlus": float(F),
            "Bell S Value": float(S)
        }

    # Print final results
    for st, val in results.items():
        print(f"=== State: {st} ===")
        for k, v in val.items():
            print(f"  {k}: {v}")
        print()
        # You can expand to see how different states yield different detection stats.

if __name__ == "__main__":
    main()


ValueError: matrix shape (4, 1) is not square.

In [10]:
import numpy as np
from qutip import basis, ket2dm, tensor, fidelity, qeye

###############################################################################
# 1) Define 2-qubit states as density matrices
###############################################################################
H = basis(2, 0)
V = basis(2, 1)

# |Phi+> ket
phi_plus_ket = (tensor(H, H) + tensor(V, V)).unit()
# Convert to density matrix
phi_plus = ket2dm(phi_plus_ket)  # 4x4

# rho = 0.7|Phi+><Phi+| + 0.15|HV><HV| + 0.15|VH><VH|
rho = 0.7 * phi_plus \
    + 0.15 * ket2dm(tensor(H, V)) \
    + 0.15 * ket2dm(tensor(V, H))

# |phi> = (1/sqrt(5))|HH> + (2/sqrt(5))|VV>, then to DM
phi_ket = (1/np.sqrt(5)*tensor(H, H) + 2/np.sqrt(5)*tensor(V, V)).unit()
phi = ket2dm(phi_ket)

# rho' = 0.7|phi><phi| + 0.15|HV><HV| + 0.15|VH><VH|
rho_prime = 0.7*phi \
    + 0.15*ket2dm(tensor(H, V)) \
    + 0.15*ket2dm(tensor(V, H))

# Put them in a dictionary, all are now 4x4 density matrices
all_states = {
    "Phi+": phi_plus,
    "rho": rho,
    "phi": phi,
    "rho_prime": rho_prime
}

###############################################################################
# 2) Generate entangled-pair timestamps (shared for Alice & Bob)
###############################################################################
def generate_pair_timestamps(pair_rate, total_time):
    """
    Generate an array of 'pair' arrival times in [0, total_time],
    from a Poisson process with average rate = pair_rate.
    """
    n_pairs = np.random.poisson(pair_rate * total_time)
    return np.random.rand(n_pairs) * total_time

###############################################################################
# 3a) Detect real photons on each side from the shared pair times
###############################################################################
def detect_photons_from_pairs(pair_times, eff_detector, dead_time, total_time):
    """
    For each pair time in 'pair_times':
      - We attempt detection with probability eff_detector
      - We skip arrivals if the detector is 'dead' from a previous detection
        for 'dead_time' seconds.

    Returns: np.array of detection times (photons)
    """
    times_sorted = np.sort(pair_times)
    detect_times = []
    last_det_t = -np.inf
    for t in times_sorted:
        if t - last_det_t < dead_time:
            continue
        # attempt detection
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

###############################################################################
# 3b) Generate & detect dark counts on each side separately
###############################################################################
def generate_dark_counts(dark_rate, eff_detector, dead_time, total_time):
    """
    Simulate dark-count arrivals at 'dark_rate' (Poisson process),
    then apply detection logic (eff_detector + dead_time).
    """
    n_dark = np.random.poisson(dark_rate * total_time)
    dark_arrivals = np.sort(np.random.rand(n_dark) * total_time)

    detect_times = []
    last_det_t = -np.inf
    for t in dark_arrivals:
        if t - last_det_t < dead_time:
            continue
        # attempt detection
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

###############################################################################
# 4) Counting coincidences with a 1 ns window, separating photon/dark
###############################################################################
def count_coincidences(a_photon, a_dark, b_photon, b_dark, coinc_window=1e-9):
    """
    Return a dict with number of coincidences in each category:
      - photon-photon
      - photon-dark
      - dark-photon
      - dark-dark
    """
    cdict = {}
    cdict["photon-photon"] = _count_coincidences_2arr(a_photon, b_photon, coinc_window)
    cdict["photon-dark"]   = _count_coincidences_2arr(a_photon, b_dark, coinc_window)
    cdict["dark-photon"]   = _count_coincidences_2arr(a_dark, b_photon, coinc_window)
    cdict["dark-dark"]     = _count_coincidences_2arr(a_dark, b_dark, coinc_window)
    return cdict

def _count_coincidences_2arr(times1, times2, window):
    """
    Two-pointer approach to count how many (t1,t2) differ by <= window.
    """
    i, j = 0, 0
    n_coinc = 0
    while i < len(times1) and j < len(times2):
        dt = times1[i] - times2[j]
        if abs(dt) <= window:
            n_coinc += 1
            i += 1
            j += 1
        elif dt < 0:
            i += 1
        else:
            j += 1
    return n_coinc

###############################################################################
# 5) Compute CHSH S-value from density matrices
###############################################################################
from qutip import qeye

def projector(theta):
    """ Projector onto cos(theta)|H> + sin(theta)|V> for 1 qubit. """
    state = np.cos(theta)*H + np.sin(theta)*V
    return ket2dm(state.unit())

def compute_expectation(rho_2q, theta_a, theta_b):
    """
    E(a, b) = (Prob(++ or --) - Prob(+- or -+)).
    We'll define A_plus, B_plus, etc. and compute directly.
    """
    A_plus = projector(theta_a)
    B_plus = projector(theta_b)

    A_minus = qeye(2) - A_plus
    B_minus = qeye(2) - B_plus

    P_pp = tensor(A_plus, B_plus)         # ++
    P_mm = tensor(A_minus, B_minus)       # --
    P_pm = tensor(A_plus, B_minus)        # +-
    P_mp = tensor(A_minus, B_plus)        # -+

    p_pp = (P_pp * rho_2q).tr().real
    p_mm = (P_mm * rho_2q).tr().real
    p_pm = (P_pm * rho_2q).tr().real
    p_mp = (P_mp * rho_2q).tr().real

    return (p_pp + p_mm) - (p_pm + p_mp)

###############################################################################
# 6) Main simulation routine
###############################################################################
def main():
    # Parameters
    PAIR_RATE   = 15000    # entangled pairs per second
    DARK_RATE   = 1000     # dark counts/s
    EFF_DET     = 0.10     # 10% detection efficiency
    DEAD_TIME   = 4e-6     # 4 microseconds
    T_RUN       = 30       # run for 30 seconds
    COINC_WIN   = 1e-9     # 1 ns

    results = {}

    for state_name, rho_2q in all_states.items():
        #--- (A) Generate entangled-pair arrival times
        pair_times = generate_pair_timestamps(PAIR_RATE, T_RUN)

        #--- (B) Local detection for real photons
        alice_photon_t = detect_photons_from_pairs(pair_times, EFF_DET, DEAD_TIME, T_RUN)
        bob_photon_t   = detect_photons_from_pairs(pair_times, EFF_DET, DEAD_TIME, T_RUN)

        #--- (C) Dark counts on each side
        alice_dark_t = generate_dark_counts(DARK_RATE, EFF_DET, DEAD_TIME, T_RUN)
        bob_dark_t   = generate_dark_counts(DARK_RATE, EFF_DET, DEAD_TIME, T_RUN)

        #--- (D) Merge times if you want total detections
        alice_all = np.sort(np.concatenate([alice_photon_t, alice_dark_t]))
        bob_all   = np.sort(np.concatenate([bob_photon_t, bob_dark_t]))

        #--- (E) Coincidence analysis with separate categories
        cdict = count_coincidences(alice_photon_t, alice_dark_t,
                                   bob_photon_t, bob_dark_t,
                                   coinc_window=COINC_WIN)
        # total coincidence (any detection)
        def _count_any_coinc(a_times, b_times):
            return _count_coincidences_2arr(a_times, b_times, COINC_WIN)

        total_coinc = _count_any_coinc(alice_all, bob_all)
        coincidence_rate = total_coinc / T_RUN

        #--- (F) Detection rates
        alice_rate = len(alice_all) / T_RUN
        bob_rate   = len(bob_all)   / T_RUN

        #--- (G) Fidelity vs. |Phi+>
        from qutip import fidelity
        F = fidelity(rho_2q, phi_plus)

        #--- (H) Bell S value (theoretical from the DM)
        # Typical angles for CHSH:
        E1 = compute_expectation(rho_2q, 0,       np.pi/8)
        E2 = compute_expectation(rho_2q, 0,       3*np.pi/8)
        E3 = compute_expectation(rho_2q, np.pi/4, np.pi/8)
        E4 = compute_expectation(rho_2q, np.pi/4, 3*np.pi/8)
        S = E1 - E2 + E3 + E4

        results[state_name] = {
            "Alice total rate (counts/s)": alice_rate,
            "Bob total rate (counts/s)":   bob_rate,

            "Alice photon rate": len(alice_photon_t)/T_RUN,
            "Alice dark rate":   len(alice_dark_t)/T_RUN,
            "Bob photon rate":   len(bob_photon_t)/T_RUN,
            "Bob dark rate":     len(bob_dark_t)/T_RUN,

            "Coincidences (ph-ph)":   cdict["photon-photon"],
            "Coincidences (ph-dark)": cdict["photon-dark"],
            "Coincidences (dark-ph)": cdict["dark-photon"],
            "Coincidences (dark-dark)": cdict["dark-dark"],

            "Total coincidences":       total_coinc,
            "Coincidence rate (counts/s)": coincidence_rate,

            "Fidelity vs. PhiPlus": float(F),
            "Bell S value": float(S)
        }

    # Print results
    for st, val in results.items():
        print(f"=== State: {st} ===")
        for k, v in val.items():
            print(f"  {k}: {v}")
        print()

if __name__ == "__main__":
    main()


=== State: Phi+ ===
  Alice total rate (counts/s): 1597.6
  Bob total rate (counts/s): 1588.8
  Alice photon rate: 1494.7
  Alice dark rate: 102.9
  Bob photon rate: 1488.4666666666667
  Bob dark rate: 100.33333333333333
  Coincidences (ph-ph): 4516
  Coincidences (ph-dark): 0
  Coincidences (dark-ph): 0
  Coincidences (dark-dark): 0
  Total coincidences: 4516
  Coincidence rate (counts/s): 150.53333333333333
  Fidelity vs. PhiPlus: 1.0000000052683549
  Bell S value: 2.82842712474619

=== State: rho ===
  Alice total rate (counts/s): 1599.1666666666667
  Bob total rate (counts/s): 1599.9666666666667
  Alice photon rate: 1495.6666666666667
  Alice dark rate: 103.5
  Bob photon rate: 1502.3666666666666
  Bob dark rate: 97.6
  Coincidences (ph-ph): 4535
  Coincidences (ph-dark): 0
  Coincidences (dark-ph): 0
  Coincidences (dark-dark): 0
  Total coincidences: 4535
  Coincidence rate (counts/s): 151.16666666666666
  Fidelity vs. PhiPlus: 0.8366600265340758
  Bell S value: 1.555634918610404